In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
import ftfy
import textacy
import csv




In [28]:
PATH_TO_DATASETS=Path('../../hedwig-data/datasets')

def df_to_hedwig_tsv(df, dsname, outfilename, num_labels_in_col, label_cols:int=[0], text_col:int=1):
    def to_tsv(outfpath, labels:list, texts:list):
        with open(outfpath, 'w', newline='') as tsvfile:
            writer = csv.writer(tsvfile, delimiter='\t')
            for label, text in zip(labels, texts):
                writer.writerow([label, text])
    df.iloc[:,1] = df.iloc[:,1].str.replace("\n", " ")
    df.iloc[:,1]=df.iloc[:,1].apply(ftfy.fix_text)
    df.iloc[:,1]=df.iloc[:,1].apply(lambda text: textacy.preprocess_text(text,lowercase=False, no_currency_symbols=True,no_urls=True,no_emails=True,no_phone_numbers=True,no_accents=True,no_contractions=False))
    df.iloc[:,1] = df.iloc[:,1].apply(lambda text: text.replace('\"', ''))
    df.iloc[:,1] = df.iloc[:,1].apply(lambda text: text.replace('"', ''))
    df.iloc[:,1] = df.iloc[:,1].apply(lambda text: ' '.join(text.split()).strip())
    df.iloc[:,1]=df.iloc[:,1].apply(ftfy.fix_text)
    df.iloc[:,0] = df.apply(lambda row: ''.join([str(lbl) for lbl in row[label_cols]]), axis=1)
    df = df.iloc[:,[0, 1]]
    df.iloc[:,0]=df.iloc[:,0].astype('str')
    df.iloc[:,0]=df.iloc[:,0].apply(
        lambda x: x if len(x) == num_labels_in_col else ''.join(
            ['0' for i in range(num_labels_in_col-len(x))]
        )+str(x)
    )
    dspath=PATH_TO_DATASETS/dsname
    outfpath = dspath/outfilename
    to_tsv(outfpath, df.iloc[:,0].tolist(), df.iloc[:,1].tolist())
    return df


In [29]:
df = pd.read_csv('../../hedwig-data/datasets/MBTI/dev.csv')
tdf=df_to_hedwig_tsv(df, dsname="MBTI", outfilename='dev.tsv', num_labels_in_col=4)

46578


,label,text
0,1000,@ USER @ USER @ USER would be great or @ USER ...
1,1000,sarah manning calls for @ USER : the pomplamoo...
2,1000,i 'm a victim of sexual assault & amp ; the la...
3,1000,! ! ! ! ! hill staffers walked out today in pr...
4,1000,with one beautiful hashtag ( via @ USER ) @ UR...


In [31]:
df = pd.read_csv('../../hedwig-data/datasets/MBTI/test.csv')
tdf=df_to_hedwig_tsv(df, dsname="MBTI", outfilename='test.tsv', num_labels_in_col=4)

In [32]:
df = pd.read_csv('../../hedwig-data/datasets/MBTI/train.csv')
tdf=df_to_hedwig_tsv(df, dsname="MBTI", outfilename='train.tsv', num_labels_in_col=4)